In [1]:
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-..."

In [2]:
from semantic_router.llms import OpenAILLM

llm = OpenAILLM(name="gpt-4o-2024-08-06")

/Users/jamesbriggs/opt/anaconda3/envs/graphai/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/Users/jamesbriggs/opt/anaconda3/envs/graphai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pydantic import BaseModel, Field


class Search(BaseModel):
    query: str = Field(description="Search query for internet information")

class Memory(BaseModel):
    query: str = Field(description="Self-directed query to search information from your long term memory")

In [4]:
import ast
import openai
from graphai import router, node
from semantic_router.schema import Message


@node(start=True)
def node_start(input: dict):
    """Descriptive string for the node."""
    print("node_a")
    return {"input": input}


@router
def node_router(input: dict):
    print("node_router")
    query = input["query"]
    messages = [
        Message(
            role="system",
            content="""You are a helpful assistant. Select the best route to answer the user query. ONLY choose one function.""",
        ),
        Message(role="user", content=query),
    ]
    response = llm(
        messages=messages,
        function_schemas=[
            openai.pydantic_function_tool(Search),
            openai.pydantic_function_tool(Memory),
        ],
    )
    choice = ast.literal_eval(response)[0]
    print("choice", choice)
    return {
        "choice": choice["function_name"].lower(),
        "input": {**input, **choice["arguments"]},
    }


@node(stream=True)
def memory(input: dict, callback = None):
    print("memory")
    query = input["query"]
    callback(query)
    print("memory query", query)
    return {"input": {"text": "The user is in Bali right now.", **input}}


@node(stream=True)
def search(input: dict, callback = None):
    print("search")
    query = input["query"]
    callback(query)
    print("search query", query)
    return {
        "input": {
            "text": "The most famous photo spot in Bali is the Uluwatu Temple.",
            **input,
        }
    }


@node(stream=True)
def llm_node(input: dict, callback = None):
    print("llm_node")
    chat_history = [
        Message(role=message["role"], content=message["content"])
        for message in input["chat_history"]
    ]

    messages = [
        Message(role="system", content="""You are a helpful assistant."""),
        *chat_history,
        Message(
            role="user",
            content=(
                f"Response to the following query from the user: {input['query']}\n"
                "Here is additional context. You can use it to answer the user query. "
                f"But do not directly reference it: {input.get('text', '')}."
            ),
        ),
    ]
    response = llm(messages=messages)
    return {"output": response}


@node(end=True)
def node_end(input: dict, callback = None):
    """Descriptive string for the node."""
    print("node_end")
    return {"output": input["output"]}

2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=False
2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=False
2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=True
2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=True
2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=True
2024-09-07 19:54:39 INFO semantic_router.utils.logger JB TEMP !!!: stream=False


In [5]:
from graphai import Graph

graph = Graph()

In [6]:
for node_fn in [node_start, memory, search, llm_node, node_end]:
    graph.add_node(node_fn)

# add the router
graph.add_router(
    source=node_start,
    router=node_router,
    destinations=[memory, search]
)

# build the certain edges
graph.add_edge(source=node_start, destination=node_router)
graph.add_edge(source=memory, destination=llm_node)
graph.add_edge(source=search, destination=llm_node)
graph.add_edge(source=llm_node, destination=node_end)

#graph.visualize()

In [7]:
graph.compile()

Graph compiled successfully.


In [8]:
chat_history = []

In [9]:
async def chat(query: str):
    chat_history.append({"role": "user", "content": query})
    response = await graph.async_execute(
        input={"input": {"query": query, "chat_history": chat_history}}
    )
    chat_history.append({"role": "assistant", "content": response["output"]})
    return response["output"]

In [10]:
await chat("do you remember where I am?")

2024-09-07 19:54:41 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='node_start'
2024-09-07 19:54:41 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:41 INFO semantic_router.utils.logger TEST 2: input={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:41 INFO semantic_router.utils.logger JB TEMP !!!: args_dict={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:41 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='node_router'
2024-09-07 19:54:41 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:

node_a
node_router
choice {'function_name': 'Memory', 'arguments': {'query': 'user location'}}


ValueError: No callback provided to graph. Please add it via `.add_callback`.

In [12]:
from graphai.callback import Callback

callback = Callback()

graph.add_callback(callback)

In [13]:
callback

In [14]:
await chat("do you remember where I am?")

2024-09-07 19:54:52 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='node_start'
2024-09-07 19:54:52 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:52 INFO semantic_router.utils.logger TEST 2: input={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:52 INFO semantic_router.utils.logger JB TEMP !!!: args_dict={'input': {'query': 'do you remember where I am?', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:52 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='node_router'
2024-09-07 19:54:52 INFO seman

node_a
node_router


2024-09-07 19:54:53 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='memory'
2024-09-07 19:54:53 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'user location', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}]}}
2024-09-07 19:54:53 INFO semantic_router.utils.logger JB TEMP !!!: stream=True
2024-09-07 19:54:53 INFO semantic_router.utils.logger JB TEMP !!!: callback=<graphai.callback.Callback object at 0x11f030550>
2024-09-07 19:54:53 INFO semantic_router.utils.logger TEST 2: input={'input': {'query': 'user location', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}]}, 'callback': <graphai.callback.Callback object at 0x11f030550>}
2024-09-07 19:54:53 INFO semantic_router.utils.logger JB TEMP !!!: args_dict={'input': {'query': 'user location', 'chat_history': [{'role': 'user', 'content

choice {'function_name': 'Memory', 'arguments': {'query': 'user location'}}
memory
memory query user location
llm_node


"You're currently in Bali. How can I assist you further?"

In [15]:
query = "tell me a long story"
response = await graph.async_execute(
    input={"input": {"query": query, "chat_history": chat_history}}
)

async for token in callback.aiter():
    print(token)

2024-09-07 19:54:55 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='node_start'
2024-09-07 19:54:55 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'tell me a long story', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'assistant', 'content': "You're currently in Bali. How can I assist you further?"}]}}
2024-09-07 19:54:55 INFO semantic_router.utils.logger TEST 2: input={'input': {'query': 'tell me a long story', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'assistant', 'content': "You're currently in Bali. How can I assist you further?"}]}}
2024-09-07 19:54:55 INFO semantic_router.utils.logger JB TEMP !!!: args_dict={'input': {'query': 'tell me a long story', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'cont

node_a
node_router


2024-09-07 19:54:56 INFO semantic_router.utils.logger JB TEMP !!!: func.__name__='memory'
2024-09-07 19:54:56 INFO semantic_router.utils.logger TEST: input={'input': {'query': 'long story', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'assistant', 'content': "You're currently in Bali. How can I assist you further?"}]}}
2024-09-07 19:54:56 INFO semantic_router.utils.logger JB TEMP !!!: stream=True
2024-09-07 19:54:56 INFO semantic_router.utils.logger JB TEMP !!!: callback=<graphai.callback.Callback object at 0x11f030550>
2024-09-07 19:54:56 INFO semantic_router.utils.logger TEST 2: input={'input': {'query': 'long story', 'chat_history': [{'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'user', 'content': 'do you remember where I am?'}, {'role': 'assistant', 'content': "You're currently in Bali. How can I assist you further?"}]}, 'callback': <graphai.callback.Callback 

choice {'function_name': 'Memory', 'arguments': {'query': 'long story'}}
memory
memory query long story
llm_node


2024-09-07 19:54:57 INFO semantic_router.utils.logger [*]  Stream Started


<graphai:start:memory>
user location
<graphai:end:memory>
<graphai:start:llm_node>
<graphai:end:llm_node>
<graphai:start:memory>
long story
<graphai:end:memory>
<graphai:start:llm_node>
<graphai:end:llm_node>
